## Initilaization

In [1]:
import numpy as np
import glob2
import datetime
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# from notebooks.demo.Open_ISAS_Grid import travel_time
#
# from notebooks.demo.dev_asso_manual import model
from utils.detection.association import load_detections
from utils.detection.association import compute_grids
from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.sound_model.spherical_sound_model import HomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.detection.association import compute_candidates, association_is_new, update_valid_grid, update_results

In [2]:
# paths
CATALOG_PATH = "/media/rsafran/CORSAIR/OHASISBIO/recensement_stations_OHASISBIO_RS.csv"
# DETECTIONS_DIR = "/media/rsafran/CORSAIR/temp/2018"
DETECTIONS_DIR = "/home/rsafran/Bureau/tissnet/2018"
ASSOCIATION_OUTPUT_DIR = "../../../data/detection/association"

# Detections loading parameters
RELOAD_DETECTIONS = False # if False, load files called "detections.npy" and "detections_merged.npy" containing everything instead of the raw detection output. Leave at True by default
MIN_P_TISSNET_PRIMARY = 0.8  # min probability of browsed detections
MIN_P_TISSNET_SECONDARY = 0.6  # min probability of detections that can be associated with the browsed one
MERGE_DELTA_S = 10 # threshold below which we consider two events should be merged
MERGE_DELTA = datetime.timedelta(seconds=MERGE_DELTA_S)

REQ_CLOSEST_STATIONS = 0  # The REQ_CLOSEST_STATIONS th closest stations will be required for an association to be valid

# sound model definition
SOUND_MODEL = HomogeneousSoundModel(sound_speed=1485.5)

# association running parameters
RUN_ASSOCIATION = True # set to False to load previous associations without processing it again
SAVE_PATH_ROOT = None  # change this to save the grids as figures, leave at None by default
NCPUS = 20  # nb of CPUs used

In [3]:
STATIONS = StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()
DETECTIONS_DIR_NAME = DETECTIONS_DIR.split("/")[-1]

if RELOAD_DETECTIONS:
    det_files = [f for f in glob2.glob(DETECTIONS_DIR + "/*") if Path(f).is_file()]
    DETECTIONS, DETECTIONS_MERGED = load_detections(det_files, STATIONS, DETECTIONS_DIR, MIN_P_TISSNET_PRIMARY, MIN_P_TISSNET_SECONDARY, MERGE_DELTA)
else:
    DETECTIONS = np.load(f"{DETECTIONS_DIR}/cache/detections.npy", allow_pickle=True).item()
    DETECTIONS_MERGED = np.load(f"{DETECTIONS_DIR}/cache/detections_merged.npy", allow_pickle=True)

STATIONS = [s for s in DETECTIONS.keys()]
FIRSTS_DETECTIONS = {s : DETECTIONS[s][0,0] for s in STATIONS}
LASTS_DETECTIONS = {s : DETECTIONS[s][-1,0] for s in STATIONS}

In [4]:
print(vars(STATIONS[0]))
print(STATIONS[0].depth)
print(STATIONS[0].name)
print(STATIONS[0].get_pos(include_depth=True))


{'manager': None, 'path': '/media/rsafran/CORSAIR/OHASISBIO/2018/ELAN', 'name': 'ELAN', 'date_start': datetime.datetime(2018, 1, 16, 17, 34, 34), 'date_end': datetime.datetime(2019, 1, 23, 7, 17, 40), 'lat': -56.4602, 'lon': 62.976, 'dataset': '2018', 'other_kwargs': {'dataset': 2018, 'station_name': 'ELAN', 'date_start': Timestamp('2018-01-16 17:34:34'), 'date_end': Timestamp('2019-01-23 07:17:40'), 'lat': -56.4602, 'lon': 62.976, 'hydrophone_serial': 580014.0, 'sensitivity': -163.8, 'depth': 1020, 'clock_drift_ppm': -0.0222, 'gps_sync ': 'ok', 'batterie': nan, 'path': '/media/rsafran/CORSAIR/OHASISBIO/2018/ELAN'}}
0.0
ELAN
[-56.4602, 62.976, 0.0]


In [5]:
LAT_BOUNDS = [-60, 5]
LON_BOUNDS = [35, 120]
GRID_SIZE = 350  # number of points along each axis

(PTS_LAT, PTS_LON, STATION_MAX_TRAVEL_TIME, GRID_STATION_TRAVEL_TIME,
 GRID_STATION_COUPLE_TRAVEL_TIME, GRID_TOLERANCE) = compute_grids(LAT_BOUNDS, LON_BOUNDS, GRID_SIZE, SOUND_MODEL, STATIONS, pick_uncertainty=2, sound_speed_uncertainty=1)

Grid tolerance of 11.85s


## Association

In [ ]:
print("starting association")

OUT_DIR = f"{ASSOCIATION_OUTPUT_DIR}/grids/{DETECTIONS_DIR_NAME}"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
OUT_FILE = f"{OUT_DIR}/s_{LAT_BOUNDS[0]}-{LAT_BOUNDS[1]},{LON_BOUNDS[0]}-{LON_BOUNDS[1]},{GRID_SIZE},{MIN_P_TISSNET_PRIMARY},{MIN_P_TISSNET_SECONDARY}.npy".replace(" ","")

association_hashlist = set()
associations = {}

def process_detection(arg):
    detection, local_association_hashlist = arg
    local_association = {}
    date1, p1, s1 = detection
    save_path = SAVE_PATH_ROOT
    if save_path is not None:
        save_path = f'{save_path}/{s1.name}-{date1.strftime("%Y%m%d_%H%M%S")}'
        Path(save_path).mkdir(parents=True, exist_ok=True)

    # list all other stations and sort them by distance from s1
    other_stations = np.array([s2 for s2 in STATIONS if s2 != s1
                               and date1 + datetime.timedelta(seconds=4*GRID_TOLERANCE) > FIRSTS_DETECTIONS[s2]
                               and date1 - datetime.timedelta(seconds=4*GRID_TOLERANCE) < LASTS_DETECTIONS[s2]])
    other_stations = other_stations[np.argsort([STATION_MAX_TRAVEL_TIME[s1][s2] for s2 in other_stations])]

    # given the detection date1 occurred on station s1, list all the detections of other stations that may be generated by the same source event
    current_association = {s1:date1}
    candidates =  compute_candidates(other_stations, current_association, DETECTIONS, STATION_MAX_TRAVEL_TIME, MERGE_DELTA_S)

    # update the list of other stations to only include the ones having at least a candidate detection
    other_stations = [s for s in other_stations if len(candidates[s]) > 0]

    if len(other_stations) < 2:
        return local_association, local_association_hashlist

    # define the recursive browsing function (that is responsible for browsing the search space of associations for s1-date1)
    def backtrack(station_index, current_association, valid_grid, associations, save_path):
        if station_index == len(other_stations):
            return
        station = other_stations[station_index]

        candidates = compute_candidates([station], current_association, DETECTIONS, STATION_MAX_TRAVEL_TIME, MERGE_DELTA_S)
        for idx in candidates[station]:
            date, p = DETECTIONS[station][idx]
            if not association_is_new(current_association, date, local_association_hashlist):
                continue

            valid_grid_new, dg_new = update_valid_grid(current_association, valid_grid, station, date, GRID_STATION_COUPLE_TRAVEL_TIME, GRID_TOLERANCE, save_path, LON_BOUNDS, LAT_BOUNDS)

            valid_points_new = np.argwhere(valid_grid_new)

            if len(valid_points_new) > 0:
                current_association[station] = (date)

                if len(current_association) > 2:
                    update_results(date1, current_association, valid_points_new, local_association, GRID_STATION_COUPLE_TRAVEL_TIME)

                backtrack(station_index + 1, current_association, valid_grid_new, associations, save_path)
                del current_association[station]
        # also try without self
        if station_index >= REQ_CLOSEST_STATIONS:
            backtrack(station_index + 1, current_association, valid_grid, associations, save_path)
        return
    backtrack(0, current_association, None, associations, save_path=save_path)
    return local_association, local_association_hashlist

# main part
if RUN_ASSOCIATION:
    try:
        with ProcessPoolExecutor(NCPUS) as executor:
            futures = {executor.submit(process_detection, (det, association_hashlist)): det for det in DETECTIONS_MERGED}
            for future in tqdm(as_completed(futures), total=len(futures)):
                local_association, local_association_hashlist = future.result()
                association_hashlist = association_hashlist.union(local_association_hashlist)
                associations = associations | local_association
    finally:
        # save the associations no matter if the execution stopped properly
        np.save(OUT_FILE, associations)

## plotting density map

In [ ]:
OUT_DIR = f"{ASSOCIATION_OUTPUT_DIR}/grids/{DETECTIONS_DIR_NAME}"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
OUT_FILE = f"{OUT_DIR}/s_{LAT_BOUNDS[0]}-{LAT_BOUNDS[1]},{LON_BOUNDS[0]}-{LON_BOUNDS[1]},{GRID_SIZE},{MIN_P_TISSNET_PRIMARY},{MIN_P_TISSNET_SECONDARY}.npy".replace(" ","")
valid = np.zeros((GRID_SIZE,GRID_SIZE))

MIN_SIZE = 4

# load every npy file in the output directory and create a grid containing associations with cardinal >= 4
for f in tqdm(glob2.glob(f"{OUT_FILE[:-4]}*.npy")):
    associations = np.load(f, allow_pickle=True).item()
    for date, associations_ in associations.items():
        for (detections, valid_points) in associations_:
            if len(detections) < MIN_SIZE:
                continue
            for i, j in valid_points:
                valid[i,j] += 1

plt.figure(figsize=(15,10))
extent = (LON_BOUNDS[0], LON_BOUNDS[-1], LAT_BOUNDS[0], LAT_BOUNDS[-1])
im = plt.imshow(valid[::-1], aspect=1, cmap="inferno", extent=extent, interpolation=None)
cbar = plt.colorbar(im)
cbar.set_label('Nb of associations')

for s in STATIONS:
    p = s.get_pos()

    if p[0] > LAT_BOUNDS[1] or p[0] < LAT_BOUNDS[0] or p[1] > LON_BOUNDS[1] or p[1] < LON_BOUNDS[0]:
        print(f"Station {s.name} out of bounds")
        continue
    plt.plot(p[1], p[0], 'wx', alpha=0.75)
    plt.annotate(s.name, xy=(p[1], p[0]), xytext=(p[1]-(LON_BOUNDS[1]-LON_BOUNDS[0])/15, p[0]+(LAT_BOUNDS[1]-LAT_BOUNDS[0])/100), textcoords="data", color='w', alpha=0.9)

In [ ]:
valid = np.zeros((GRID_SIZE,GRID_SIZE))

MIN_SIZE = 4

# load every npy file in the output directory and create a grid containing associations with cardinal >= 4
for f in tqdm(glob2.glob(f"{OUT_FILE[:-4]}*.npy")):
    associations = np.load(f, allow_pickle=True).item()
    for date, associations_ in associations.items():
        for (detections, valid_points) in associations_:
            if len(detections) < MIN_SIZE:
                continue
            for i, j in valid_points:
                valid[i,j] += 1

# Create a figure with cartopy's PlateCarree projection
projection = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': projection})

# Set the extent of the map (min_lon, max_lon, min_lat, max_lat)
ax.set_extent([LON_BOUNDS[0], LON_BOUNDS[1], LAT_BOUNDS[0], LAT_BOUNDS[1]], crs=projection)

# Add natural features: land, ocean, and coastlines.
# These features will be drawn on top if the image is behind.
ax.add_feature(cfeature.LAND, facecolor='lightgray', zorder=2)
# ax.add_feature(cfeature.OCEAN, facecolor='lightblue', zorder=2)
ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=1, zorder=3)
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='black', zorder=3)

# Plot the georeferenced image.
# Set a lower zorder (e.g., 1) so that the map features drawn with higher zorders remain visible.
# Adjust alpha to add a bit of transparency if desired.
extent = (LON_BOUNDS[0], LON_BOUNDS[1], LAT_BOUNDS[0], LAT_BOUNDS[1])
im = ax.imshow(valid[::-1],
               cmap="tab20c",
               extent=extent,
               interpolation="nearest",
               origin='upper',
               transform=projection,
               zorder=1,
               alpha=1, vmax=500)

# Add a colorbar for the image.
cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.05)
cbar.set_label('Nb of associations')

# Plot station markers and add annotations using the axes methods.
for s in STATIONS:
    lat, lon = s.get_pos()
    if lat > LAT_BOUNDS[1] or lat < LAT_BOUNDS[0] or lon > LON_BOUNDS[1] or lon < LON_BOUNDS[0]:
        print(f"Station {s.name} out of bounds")
        continue
    # Plot a marker with a higher zorder so it's on top of the image
    ax.plot(lon, lat, 'wx', alpha=0.75, markersize=8, transform=projection, zorder=4)
    ax.text(lon - (LON_BOUNDS[1] - LON_BOUNDS[0]) / 15,
            lat + (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 100,
            s.name,
            color='white',
            alpha=0.9,
            transform=projection,
            zorder=4)

plt.title("Association Data with Land and Sea")
plt.show()


In [ ]:
path_asso = "/home/rsafran/PycharmProjects/toolbox/data/detection/association/grids/2018/s_-60--12.4,35-100,350,0.8,0.5.npy"
associations = np.load(path_asso, allow_pickle=True).item()

## To Dataframe

In [ ]:
import numpy as np
import pandas as pd

# Path to your saved associations file.
path_asso = "/home/rsafran/PycharmProjects/toolbox/data/detection/association/grids/2018/s_-60-5,35-120,350,0.8,0.6.npy"

# Load associations (allow_pickle=True because the file was saved with pickled Python objects).
associations = np.load(path_asso, allow_pickle=True).item()

# Flatten associations:
# For each association key and each candidate association (tuple) in its list,
# we build a row with the association key, candidate number, detection info, and grid info.
flattened_data = []

for assoc_key, candidates in associations.items():
    # Convert the key (datetime) to a string, if needed.
    assoc_key_str = str(assoc_key)

    for i, candidate in enumerate(candidates):
        # candidate is expected to be a tuple: (detections_array, grids_array)
        detections_array, grids_array = candidate

        # Convert detections array into a list of tuples
        # or a string representation if you want a summary.
        # For instance, each row of 'detections_array' is [station, datetime]
        detection_list = []
        for row in detections_array:
            # row[0] is typically the station identifier and row[1] is the detection datetime.
            detection_list.append((row[0], row[1]))

        # Similarly, convert the grid indices array to a list of lists (or string)
        grid_list = grids_array.tolist()

        flattened_data.append({
            "association_key": assoc_key_str,
            "candidate_index": i,
            "detections": detection_list,
            "grids": grid_list
        })

# Create a DataFrame from the flattened data.
df_associations = pd.DataFrame(flattened_data)

# Display the first few rows of the DataFrame.
# print(df_associations.head())

# Optionally, save the DataFrame to a CSV file.
df_associations.to_csv("associations_dataframe.csv", index=False)


In [ ]:
df_filtered  = df_associations[df_associations['detections'].apply(len) > 6]

In [ ]:
df_associations

In [ ]:
valid = np.zeros((GRID_SIZE,GRID_SIZE))

MIN_SIZE = 5

# load every npy file in the output directory and create a grid containing associations with cardinal >= 4
for grid in df_filtered['grids']:
    for valid_points in grid:
        valid[valid_points[0], valid_points[1]] += 1

# Create a figure with cartopy's PlateCarree projection
projection = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': projection})

# Set the extent of the map (min_lon, max_lon, min_lat, max_lat)
ax.set_extent([LON_BOUNDS[0], LON_BOUNDS[1], LAT_BOUNDS[0], LAT_BOUNDS[1]], crs=projection)

# Add natural features: land, ocean, and coastlines.
# These features will be drawn on top if the image is behind.
ax.add_feature(cfeature.LAND, facecolor='lightgray', zorder=2)
# ax.add_feature(cfeature.OCEAN, facecolor='lightblue', zorder=2)
ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=1, zorder=3)
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='black', zorder=3)

# Plot the georeferenced image.
# Set a lower zorder (e.g., 1) so that the map features drawn with higher zorders remain visible.
# Adjust alpha to add a bit of transparency if desired.
extent = (LON_BOUNDS[0], LON_BOUNDS[1], LAT_BOUNDS[0], LAT_BOUNDS[1])
im = ax.imshow(valid[::-1],
               cmap="tab20c",
               extent=extent,
               interpolation="nearest",
               origin='upper',
               transform=projection,
               zorder=1,
               alpha=1, vmax=5)

# Add a colorbar for the image.
cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.05)
cbar.set_label('Nb of associations')

# Plot station markers and add annotations using the axes methods.
for s in STATIONS:
    lat, lon = s.get_pos()
    if lat > LAT_BOUNDS[1] or lat < LAT_BOUNDS[0] or lon > LON_BOUNDS[1] or lon < LON_BOUNDS[0]:
        print(f"Station {s.name} out of bounds")
        continue
    # Plot a marker with a higher zorder so it's on top of the image
    ax.plot(lon, lat, 'wx', alpha=0.75, markersize=8, transform=projection, zorder=4)
    ax.text(lon - (LON_BOUNDS[1] - LON_BOUNDS[0]) / 15,
            lat + (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 100,
            s.name,
            color='white',
            alpha=0.9,
            transform=projection,
            zorder=4)

plt.title("Association Data min {}".format(MIN_SIZE))
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

# Configuration matching your original association parameters
LAT_BOUNDS = [-60, 5]
LON_BOUNDS = [35, 120]
GRID_SIZE = 350

# Calculate grid coordinates (replicating your original grid setup)
PTS_LAT = np.linspace(LAT_BOUNDS[0], LAT_BOUNDS[1], GRID_SIZE)
PTS_LON = np.linspace(LON_BOUNDS[0], LON_BOUNDS[1], GRID_SIZE)

def grid_index_to_coords(indices):
    """Convert grid indices to (lat, lon) coordinates"""
    return [(PTS_LAT[i], PTS_LON[j]) for i, j in indices]

# Load associations
path_asso = "/home/rsafran/PycharmProjects/toolbox/data/detection/association/grids/2018/s_-60-5,35-120,350,0.8,0.6.npy"
associations = np.load(path_asso, allow_pickle=True).item()

flattened_data = []

for assoc_key, candidates in associations.items():
    # Convert numpy datetime64 to Python datetime
    event_time = pd.to_datetime(assoc_key).to_pydatetime()

    for candidate_id, (detections_array, grids_array) in enumerate(candidates):
        # Process detections
        detections = []
        for station, dt_np64 in detections_array:
            dt = pd.to_datetime(dt_np64).to_pydatetime()
            detections.append({
                "station": station,
                "detection_time": dt,
                "travel_time": (event_time - dt).total_seconds()
            })

        # Process grid points
        grid_points = grid_index_to_coords(grids_array)

        flattened_data.append({
            "event_time": event_time,
            "candidate_id": candidate_id,
            "num_stations": len(detections),
            "stations": [d["station"] for d in detections],
            "detection_times": [d["detection_time"] for d in detections],
            "travel_times": [d["travel_time"] for d in detections],
            "grid_points": grid_points,
            "num_grid_points": len(grid_points),
            "median_lat": np.median([p[0] for p in grid_points]),
            "median_lon": np.median([p[1] for p in grid_points])
        })

# Create DataFrame
df = pd.DataFrame(flattened_data)

# Add duration statistics
df["detection_time_span"] = df["detection_times"].apply(
    lambda x: (max(x) - min(x)).total_seconds()
)

print(f"Loaded {len(df)} associations with:")
print(f"- Median stations per event: {df['num_stations'].median()}")
print(f"- Median grid points per event: {df['num_grid_points'].median()}")

# Save to CSV with timestamp
output_csv = f"associations_{datetime.now().strftime('%Y%m%d')}.csv"
df.to_csv(output_csv, index=False)
print(f"\nSaved to {output_csv}")

In [ ]:
# Filter high-confidence events
strong_events = df[(df.num_grid_points >= 1 )& (df.num_stations >6)]

# Plot geographic distribution
import matplotlib.pyplot as plt
plt.scatter(strong_events.median_lon, strong_events.median_lat, s=strong_events.num_stations, alpha=0.5)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

# Analyze temporal distribution
df["event_hour"] = strong_events.event_time.dt.hour
df.event_hour.hist(bins=200)

In [ ]:
strong_events.detection_times.iloc[0]

In [ ]:
import ast
import os
from datetime import timedelta
import pandas as pd
from tqdm import tqdm
from src.utils.data_reading.sound_data.sound_file_manager import DatFilesManager
from ast import literal_eval
# Configuration
DATA_ROOT = "/media/rsafran/CORSAIR/OHASISBIO/"
OUTPUT_DIR = "./event_analysis"
BUFFER = timedelta(minutes=2)
MIN_STATIONS = 7


def safe_convert_list(val):
    """Safely convert string representation of list to actual list"""
    if pd.isna(val) or val == '[]':
        return []
    try:
        return literal_eval(val)
    except (ValueError, SyntaxError):
        return [x.strip(" '") for x in val.strip('[]').split(',')]

def safe_convert_datetime(val):
    """Convert string representation of datetime list"""
    if pd.isna(val) or val == '[]':
        return []
    try:
        return pd.to_datetime(literal_eval(val))
    except:
        return [pd.to_datetime(x.strip(" '")) for x in val.strip('[]').split(',')]

def safe_convert_coords(val):
    """Convert grid points string to list of tuples"""
    if pd.isna(val) or val == '[]':
        return []
    try:
        return [tuple(map(float, pair.strip('()').split(',')))
                for pair in val.strip('[]').split('), ')]
    except:
        return []

# Custom converters for each column type
converters = {
    'stations': safe_convert_list,
    'travel_times': lambda x: [float(v) for v in safe_convert_list(x)],
    'detection_time': safe_convert_datetime,
    'grid_points': safe_convert_coords
}

# Load dataframe with proper type conversion
df = pd.read_csv(
    "associations_20250414.csv",
    parse_dates=['event_time'],
    converters=converters
)


## Signal Check

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import pandas as pd
import numpy as np
import cartopy.crs as ccrs  # Missing import
from datetime import timedelta
from src.utils.data_reading.sound_data.sound_file_manager import DatFilesManager
# Configuration matching your original association parameters
LAT_BOUNDS = [-60, 5]
LON_BOUNDS = [35, 120]
GRID_SIZE = 350

# Calculate grid coordinates (replicating your original grid setup)
PTS_LAT = np.linspace(LAT_BOUNDS[0], LAT_BOUNDS[1], GRID_SIZE)
PTS_LON = np.linspace(LON_BOUNDS[0], LON_BOUNDS[1], GRID_SIZE)

def grid_index_to_coords(indices):
    """Convert grid indices to (lat, lon) coordinates"""
    return [(PTS_LAT[i], PTS_LON[j]) for i, j in indices]


def plot_single_event(event_index, df, data_root, stations_list, lat_bounds, lon_bounds, raw=True):
    """Interactive visualization for one seismic event"""
    try:
        event = df.iloc[event_index]
    except IndexError:
        print(f"Error: Event index {event_index} out of range (0-{len(df)-1})")
        return

    print(f"\nEvent {event_index}: {event['event_time']}")
    print(f"Stations: {', '.join(event['stations'])}")
    print(f"Median location: ({event['median_lat']:.2f}°, {event['median_lon']:.2f}°)")

    # Create figure
    fig = plt.figure(figsize=(15, 10), layout='constrained')
    gs = fig.add_gridspec(2, 1, height_ratios=[3, 2])
    ax_signals = fig.add_subplot(gs[0])
    ax_map = fig.add_subplot(gs[1], projection=ccrs.PlateCarree())

    # Initialize map elements first
    ax_map.coastlines()
    ax_map.set_extent([lon_bounds[0], lon_bounds[1], lat_bounds[0], lat_bounds[1]])

    # Plot all background stations
    for s in stations_list:
        try:
            p = s.get_pos()
            if not (lat_bounds[0] <= p[0] <= lat_bounds[1]) or \
               not (lon_bounds[0] <= p[1] <= lon_bounds[1]):
                continue
            ax_map.plot(p[1], p[0], 'bx', alpha=0.5, markersize=5)
            ax_map.annotate(s.name,
                xy=(p[1], p[0]),
                xytext=(p[1] - (lon_bounds[1]-lon_bounds[0])/15,
                        p[0] + (lat_bounds[1]-lat_bounds[0])/100),
                textcoords="data",
                color='gray',
                alpha=0.5,
                fontsize=8
            )

        except Exception as e:
            print(f"Skipping station {s.name}: {str(e)}")
            continue

    # Plot event location
    ax_map.plot(event['median_lon'], event['median_lat'], 'r*', markersize=15, zorder=10)

    # Load and plot signals
    colors = plt.cm.viridis(np.linspace(0, 1, len(event['stations'])))

    for i, (full_station_name, det_time, tt) in enumerate(zip(
        event['stations'],
        event['detection_times'],
        event['travel_times'])):
        try:
            # Split station name
            dataset, station = full_station_name.split('_', 1)
            center_time = det_time
            start = center_time - timedelta(minutes=3)
            end = center_time + timedelta(minutes=3)
            # Get data manager
            managers = DatFilesManager(f"{data_root}/{dataset}/{station}")
            # Load data
            data = managers.get_segment(start, end)
            if data is None:
                print(f"No data for {station}")
                continue

            # Plot signal
            # In the signal loading/plotting section:
            SAMPLING_RATE = 240  # Hz
            SAMPLE_INTERVAL = 1/SAMPLING_RATE  # seconds

            # Create accurate time axis
            times = pd.to_datetime(start) + pd.to_timedelta(np.arange(len(data)) * SAMPLE_INTERVAL, unit='s')
                        # Compute signal energy over a sliding window
            if not raw : #Energy plot
                SAMPLING_RATE = 240  # Hz
                SAMPLE_INTERVAL = 1 / SAMPLING_RATE  # seconds

                # Create time axis
                times = pd.to_datetime(start) + pd.to_timedelta(np.arange(len(data)) * SAMPLE_INTERVAL, unit='s')
                relative_times = (times - det_time).total_seconds()
                # Compute energy envelope
                window_sec = 5.0
                window_samples = int(window_sec * SAMPLING_RATE)
                energy = np.convolve(data**2, np.ones(window_samples)/window_samples, mode='same')

                # Normalize energy
                if np.max(energy) > 0:
                    energy /= np.max(energy)

                # Vertically shift for visual separation
                offset = i * 1.5  # You can tune this value if traces are too close/far
                energy_shifted = energy + offset

                # Plot energy
                ax_signals.plot(
                    relative_times, energy_shifted,
                    label=f"{station} (ΔT={tt:.1f}s)",
                    color=colors[i],
                    alpha=0.8
                )

                # Mark detection time
                # ax_signals.axvline(relative_times, color=colors[i], linestyle='--', alpha=0.5)

            if raw : #raw signal

                # Normalize signal
                data /= np.max(abs(data))

                # Vertically shift for visual separation
                offset = i * 1.5  # You can tune this value if traces are too close/far
                data_shifted = data + offset
                relative_times = (times - det_time).total_seconds()
                ax_signals.plot(
                    relative_times, data_shifted,
                    label=f"{station} (ΔT={tt:.1f}s)",
                    color=colors[i],
                    alpha=0.7
                )
                ax_signals.axvline(0, color=colors[i], linestyle='--', alpha=0.5)

            # Plot detection station on map
            dataset, station = full_station_name.split('_', 1)
            s = next((s for s in stations_list if s.name == station), None)
            if s:
                p = s.get_pos()
                ax_map.plot(
                        p[1], p[0],
                        marker='^',
                        markersize=8,
                        color=colors[i],
                        markeredgewidth=1,
                        zorder=9)

        except Exception as e:
            print(f"Error processing {full_station_name}: {str(e)}")
            continue

    # Format plots after all data is plotted
    ax_signals.set_title(f"Event {event['event_time']}\n{len(event['stations'])} stations detected")
    # if raw :
    #     ax_signals.xaxis.set_major_formatter(DateFormatter("%H:%M:%S"))
    #     ax_signals.set_xlabel("UTC Time")
    ax_signals.set_ylabel("Amplitude")
    ax_signals.legend(loc='upper right')

    ax_map.set_title("Event Location (red star) and Detection Stations (colored triangles)")
    plt.show()


# Usage example
if __name__ == "__main__":
    # df = pd.read_csv("associations_20250414.csv.csv", parse_dates=['event_time'])  # Add your converters
    df = df[df.num_stations > 7]
    DATA_ROOT = "/media/rsafran/CORSAIR/OHASISBIO"

    while True:
        try:
            max_idx = len(df) - 1
            prompt = f"\nEnter event index (0-{max_idx}) or -1 to quit: "
            event_idx = int(input(prompt))

            if event_idx == -1:
                break
            plot_single_event(event_idx, df, DATA_ROOT, STATIONS, LAT_BOUNDS, LON_BOUNDS)
        except ValueError:
            print("Please enter a valid number")

# Least square

### Loading

In [ ]:
import numpy as np
import pandas as pd

# Path to your saved associations file.
path_asso = "/home/rsafran/PycharmProjects/toolbox/data/detection/association/grids/2018/s_-60-5,35-120,350,0.8,0.6.npy"

# Load associations (allow_pickle=True because the file was saved with pickled Python objects).
associations = np.load(path_asso, allow_pickle=True).item()

# Flatten associations:
# For each association key and each candidate association (tuple) in its list,
# we build a row with the association key, candidate number, detection info, and grid info.
flattened_data = []

for assoc_key, candidates in associations.items():
    # Convert the key (datetime) to a string, if needed.
    assoc_key_str = str(assoc_key)

    for i, candidate in enumerate(candidates):
        # candidate is expected to be a tuple: (detections_array, grids_array)
        detections_array, grids_array = candidate

        # Convert detections array into a list of tuples
        # or a string representation if you want a summary.
        # For instance, each row of 'detections_array' is [station, datetime]
        detection_list = []
        for row in detections_array:
            # row[0] is typically the station identifier and row[1] is the detection datetime.
            detection_list.append((row[0], row[1]))

        # Similarly, convert the grid indices array to a list of lists (or string)
        grid_list = grids_array.tolist()

        flattened_data.append({
            "association_key": assoc_key_str,
            "candidate_index": i,
            "detections": detection_list,
            "grids": grid_list
        })

# Create a DataFrame from the flattened data.
df_associations = pd.DataFrame(flattened_data)

# Display the first few rows of the DataFrame.
df_associations

### eval single value

In [ ]:
df_associations = df_associations[df_associations['detections'].apply(len) > 7]
df_associations.reset_index(inplace=True)

In [ ]:

row = df_associations.iloc[555]
detections = row["detections"]
grids = row["grids"]
month = pd.to_datetime(row['association_key']).month
# print(month)
def grid_index_to_coord(indices):
    """Convert grid indices to (lat, lon) coordinates"""
    i, j = indices
    return [PTS_LAT[i], PTS_LON[j]]


for [station, d_time] in detections:
    print((station, d_time))
    print(d_time.timestamp())  #d_time is a datetime object
    print(station.get_pos())
    print(station.name) #station has a method get_pos() and .name

print(grid_index_to_coord(grids[0])) #initial guess event lat and long



In [ ]:
from utils.physics.sound_model import ISAS_grid as isg
lat_bounds = LAT_BOUNDS
lon_bounds = LON_BOUNDS
PATH = "/media/rsafran/CORSAIR/ISAS/86442/field/2018"


ds = isg.load_ISAS_TS(PATH, month,lat_bounds,lon_bounds, fast=False)
# Convert datetime to seconds since some origin (e.g., min time)
arrival_times = [d_time for _, d_time in detections]
origin_time = min(arrival_times)
arrival_times_seconds = np.array([(t - origin_time).total_seconds() for t in arrival_times])
station_positions = [station.get_pos() for station, _ in detections]
grid_init_lat = [grid_index_to_coord(grid)[0] for grid in grids]
grid_init_lon = [grid_index_to_coord(grid)[1] for grid in grids]
def residuals(params, station_positions, arrival_times, ds):
    """
    Improved residuals function that accounts for travel time uncertainty.

    params: (lat, lon, t0) where t0 is in seconds
    station_positions: list/array of (lat, lon) station coordinates
    arrival_times: array of arrival times (in seconds)
    ds: dataset with sound velocity data

    Returns:
    - weighted residuals: difference between modeled and observed arrival times,
      weighted by uncertainty when available
    """
    lat, lon, t0 = params
    modeled_times = []
    uncertainties = []

    max_depth = 1000  # Or appropriate depth for your dataset

    for i, (slat, slon) in enumerate(station_positions):
        try:
            travel_time, sig_v, _ = isg.compute_travel_time(
                slat, slon, lat, lon, depth=max_depth,
                ds=ds, resolution=30, verbose=False,
                interpolate_missing=True
            )
            modeled_times.append(t0 + travel_time)
            uncertainties.append(sig_v if sig_v > 0 else 1.0)  # Default to 1.0 if uncertainty is zero
        except Exception as e:
            print(f"Error at station {i}: {slat},{slon} to {lat},{lon}: {e}")
            # Add a placeholder with high uncertainty
            modeled_times.append(arrival_times[i])  # Use observed time as placeholder
            uncertainties.append(1000.0)  # Very high uncertainty

    modeled_times = np.array(modeled_times)
    uncertainties = np.array(uncertainties)

    # Calculate weighted residuals - divide by uncertainty to weight more certain measurements higher
    weighted_residuals = (modeled_times - arrival_times) / uncertainties

    return weighted_residuals

In [ ]:
# Better initialization with bounds and handling
from scipy.optimize import least_squares
import numpy as np
from pyproj import Geod
geod = Geod(ellps="WGS84")
from concurrent.futures import ThreadPoolExecutor
from functools import partial

# Define reasonable geographic bounds based on your area of interest
lat_min, lat_max = min(LAT_BOUNDS), max(LAT_BOUNDS)
lon_min, lon_max = min(LON_BOUNDS), max(LON_BOUNDS)

# More conservative initial guess (center of first few stations)
lat0 = np.mean(grid_init_lat)
lon0 = np.mean(grid_init_lon)


# Vectorized distance calculation
def calc_distances(lon0, lat0, positions):
    return np.array([geod.inv(lon0, lat0, pos[1], pos[0])[2] for pos in positions])


# Estimate travel time to nearest station for t0 guess
distances = calc_distances(lon0, lat0, station_positions)
nearest_idx = np.argmin(distances)
nearest_lat, nearest_lon = station_positions[nearest_idx]
_, _, distance_m = geod.inv(lon0, lat0, nearest_lon, nearest_lat)

# Use a more conservative sound speed estimate 
sound_speed = 1450  # m/s
t0_guess = -distance_m / sound_speed

# Initial guess
x0 = [lat0, lon0, t0_guess]

# Add bounds to keep solution in reasonable area
bounds = ([lat_min, lon_min, -np.inf], [lat_max, lon_max, np.inf])

# Use jac='3-point' for faster Jacobian approximation
result = least_squares(
    residuals, x0,
    args=(station_positions, arrival_times_seconds, ds),
    bounds=bounds,
    loss='huber',      # or 'huber'      # tuning parameter
    method='trf',
    ftol=1e-12, xtol=1e-12, gtol=1e-12,
    verbose=1
)



In [ ]:
# Results
estimated_lat, estimated_lon, estimated_t0 = result.x
print(f"Initial guess: {lat0, lon0, t0_guess}")
print(f"Estimated source location: ({estimated_lat:.4f}, {estimated_lon:.4f})")
print(f"Estimated origin time offset: {estimated_t0:.2f} seconds")

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Ellipse
from scipy import stats
# Your results
estimated_lat, estimated_lon, estimated_t0 = result.x
initial_lat, initial_lon = lat0, lon0

# Optional: List of receiver stations (if available)
# receivers = [(lat1, lon1), (lat2, lon2), ...]

# Create figure and map projection
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.OCEAN)

ax.gridlines(draw_labels=True)

# Plot estimated source location
ax.plot(estimated_lon, estimated_lat, marker='*', color='red', markersize=15, label='Estimated Source')

# Plot initial guess (optional)
ax.plot(initial_lon, initial_lat, marker='x', color='blue', markersize=10, label='Initial Guess')

if hasattr(result, 'jac'):
    # Get the Jacobian for the lat-lon parameters (exclude time parameter)
    J = result.jac[:, :2]

    # Calculate the covariance matrix (approximation)
    residual_variance = np.sum(result.fun**2) / (len(result.fun) - len(result.x))
    covariance_matrix = residual_variance * np.linalg.inv(J.T @ J)

    # Eigenvalue decomposition to get ellipse parameters
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # Calculate ellipse parameters
    # For 95% confidence, use 2.4477 (chi-square with 2 DOF)
    confidence = 0.95
    chi2_val = stats.chi2.ppf(confidence, 2)

    # Semi-major and semi-minor axes
    a = np.sqrt(eigenvalues[1] * chi2_val)
    b = np.sqrt(eigenvalues[0] * chi2_val)

    # Angle in degrees
    angle = np.degrees(np.arctan2(eigenvectors[1, 1], eigenvectors[0, 1]))

    # Create and add the ellipse
    fac = 1
    ellipse = Ellipse(xy=(estimated_lon, estimated_lat),
                      width=fac*2 * a, height=fac*2 * b,
                      angle=angle,
                      edgecolor='red', facecolor='red', alpha=0.8,
                      transform=ccrs.PlateCarree(),
                      label=f'{int(confidence*100)}% Confidence')
    ax.add_patch(ellipse)

# Plot receiver stations (optional)
# for lat, lon in receivers:
#     ax.plot(lon, lat, marker='^', color='green', markersize=8)

# Set extent if needed (min_lon, max_lon, min_lat, max_lat)
offset = 5
ax.set_extent([estimated_lon - offset, estimated_lon + offset, estimated_lat - offset, estimated_lat + offset])

# Add legend and title
plt.legend()
plt.title('Estimated Source Location on Map')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Ellipse
from scipy import stats

def visualize_localization_result(result, station_positions, ds, lat_bounds, lon_bounds):
    """
    Visualize source localization result with error ellipse.

    Parameters:
    - result: The optimization result object from least_squares
    - station_positions: List of (lat, lon) tuples for station positions
    - ds: The ISAS dataset (for context)
    - lat_bounds, lon_bounds: The geographic boundaries for the map
    """
    # Extract the best fit parameters and stations
    estimated_lat, estimated_lon, estimated_t0 = result.x
    station_lats = [pos[0] for pos in station_positions]
    station_lons = [pos[1] for pos in station_positions]

    # Set up the map projection
    projection = ccrs.PlateCarree()
    fig, ax = plt.subplots(figsize=(12, 10), subplot_kw={'projection': projection})

    # Add map features
    ax.coastlines(resolution='50m')
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')

    # Set map boundaries with some padding
    pad = 2  # degrees
    ax.set_extent([lon_bounds[0]-pad, lon_bounds[1]+pad,
                  lat_bounds[0]-pad, lat_bounds[1]+pad])

    # Plot stations
    ax.scatter(station_lons, station_lats, color='blue', s=100, marker='^',
               transform=ccrs.PlateCarree(), label='Stations', zorder=3)

    # Plot the estimated source location
    ax.scatter([estimated_lon], [estimated_lat], color='red', s=150, marker='+',
               transform=ccrs.PlateCarree(), label='Estimated Source', zorder=4)

    # Calculate the error ellipse using the covariance matrix
    # This requires the Jacobian from the optimization result
    if hasattr(result, 'jac'):
        # Get the Jacobian for the lat-lon parameters (exclude time parameter)
        J = result.jac[:, :2]

        # Calculate the covariance matrix (approximation)
        residual_variance = np.sum(result.fun**2) / (len(result.fun) - len(result.x))
        covariance_matrix = residual_variance * np.linalg.inv(J.T @ J)

        # Eigenvalue decomposition to get ellipse parameters
        eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

        # Calculate ellipse parameters
        # For 95% confidence, use 2.4477 (chi-square with 2 DOF)
        confidence = 0.95
        chi2_val = stats.chi2.ppf(confidence, 2)

        # Semi-major and semi-minor axes
        a = np.sqrt(eigenvalues[1] * chi2_val)
        b = np.sqrt(eigenvalues[0] * chi2_val)

        # Angle in degrees
        angle = np.degrees(np.arctan2(eigenvectors[1, 1], eigenvectors[0, 1]))

        # Create and add the ellipse
        fac = 10
        ellipse = Ellipse(xy=(estimated_lon, estimated_lat),
                          width=fac*2 * a, height=fac*2 * b,
                          angle=angle,
                          edgecolor='red', facecolor='red', alpha=0.8,
                          transform=ccrs.PlateCarree(),
                          label=f'{int(confidence*100)}% Confidence')
        ax.add_patch(ellipse)

        # Add error statistics to title
        plt.title(f'Source Localization Result\n'
                 f'Estimated Position: ({estimated_lat:.4f}°, {estimated_lon:.4f}°)\n'
                 f'Time Offset: {estimated_t0:.2f} s, RMS Error: {np.sqrt(np.mean(result.fun**2)):.2f} s')
    else:
        # If no Jacobian, just add basic title
        plt.title(f'Source Localization Result\n'
                 f'Estimated Position: ({estimated_lat:.4f}°, {estimated_lon:.4f}°)')

    # Add labels and legend
    for i, (lon, lat) in enumerate(zip(station_lons, station_lats)):
        ax.text(lon + 0.1, lat + 0.1, f'Station {i+1}', transform=ccrs.PlateCarree(),
                fontsize=9, horizontalalignment='left', verticalalignment='bottom')

    ax.legend(loc='lower right')

    # Add a scale bar if desired
    # ax.gridlines(draw_labels=True)

    plt.tight_layout()
    return fig, ax

# Call this function with your result
fig, ax = visualize_localization_result(result, station_positions, ds, LAT_BOUNDS, LON_BOUNDS)

plt.savefig('source_localization_result.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats

def analyze_residuals(result, station_positions, arrival_times, ds):
    """
    Analyze residuals to identify potential outliers in the data

    Parameters:
    - result: optimization result from least_squares
    - station_positions: list of (lat, lon) tuples for stations
    - arrival_times: observed arrival times in seconds
    - ds: ISAS dataset

    Returns:
    - residuals: array of time residuals for each station
    - is_outlier: boolean array indicating potential outliers
    """
    # Extract estimated parameters
    estimated_lat, estimated_lon, estimated_t0 = result.x

    # Calculate modeled arrival times for each station
    modeled_times = []
    station_ids = []

    for i, (slat, slon) in enumerate(station_positions):
        try:
            travel_time, _, _ = isg.compute_travel_time(
                slat, slon, estimated_lat, estimated_lon,
                depth=400, ds=ds, resolution=20,
                verbose=False, interpolate_missing=True
            )
            modeled_times.append(estimated_t0 + travel_time)
            station_ids.append(i)
        except Exception as e:
            print(f"Error calculating travel time for station {i}: {e}")

    # Convert to numpy arrays
    modeled_times = np.array(modeled_times)
    observed_times = np.array([arrival_times[i] for i in station_ids])

    # Calculate residuals (modeled - observed)
    time_residuals = modeled_times - observed_times

    # Identify outliers using Modified Z-score method
    # This is more robust than standard Z-score for small sample sizes
    median_residual = np.median(time_residuals)
    mad = np.median(np.abs(time_residuals - median_residual))  # Median Absolute Deviation

    # MAD needs to be scaled to be comparable to standard deviation
    mad_constant = 1.4826  # for normal distribution
    modified_z_scores = mad_constant * (time_residuals - median_residual) / (mad + 1e-8)

    # Define outlier threshold (typically 3.5 or higher)
    outlier_threshold = 3.5
    is_outlier = np.abs(modified_z_scores) > outlier_threshold

    # Print summary
    print("\nResidual Analysis:")
    print("-" * 50)
    print(f"{'Station':8} {'Residual (s)':12} {'Modified Z':10} {'Outlier':8}")
    print("-" * 50)

    for i, station_id in enumerate(station_ids):
        outlier_mark = "YES" if is_outlier[i] else "NO"
        print(f"{station_id:8d} {time_residuals[i]:12.2f} {modified_z_scores[i]:10.2f} {outlier_mark:8}")

    print("-" * 50)
    print(f"RMS Error: {np.sqrt(np.mean(time_residuals**2)):.2f} seconds")
    print(f"Median Absolute Deviation: {mad:.2f} seconds")

    return time_residuals, is_outlier, station_ids

def visualize_residuals(result, station_positions, arrival_times, ds, lat_bounds, lon_bounds):
    """
    Visualize residuals and identify outliers on a map
    """
    # Calculate residuals and identify outliers
    residuals, is_outlier, included_station_ids = analyze_residuals(
        result, station_positions, arrival_times, ds
    )

    # Extract estimated source location
    estimated_lat, estimated_lon, estimated_t0 = result.x

    # Set up the map
    projection = ccrs.PlateCarree()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8),
                                subplot_kw={'projection': projection})

    # --- First subplot: Map with stations and source ---
    ax1.coastlines(resolution='50m')
    ax1.add_feature(cfeature.LAND, facecolor='lightgray')
    ax1.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax1.gridlines(draw_labels=True)

    # Set map boundaries with padding
    pad = 2  # degrees
    ax1.set_extent([lon_bounds[0]-pad, lon_bounds[1]+pad,
                   lat_bounds[0]-pad, lat_bounds[1]+pad])

    # Plot stations with color based on residual
    # Convert residuals to colormap values
    norm = plt.Normalize(vmin=-np.max(np.abs(residuals)), vmax=np.max(np.abs(residuals)))

    # Plot stations
    for i, station_id in enumerate(included_station_ids):
        slat, slon = station_positions[station_id]

        # Marker properties based on whether it's an outlier
        marker_size = 150 if is_outlier[i] else 100
        marker_edge = 'black' if is_outlier[i] else None
        marker_width = 2 if is_outlier[i] else 0

        # Color based on residual value
        color = plt.cm.RdBu_r(norm(residuals[i]))

        # Add station marker
        ax1.scatter([slon], [slat], color=color, s=marker_size, marker='^',
                   edgecolor=marker_edge, linewidth=marker_width,
                   transform=ccrs.PlateCarree())

        # Add station label
        ax1.text(slon + 0.1, slat + 0.1, f'Station {station_id}',
                transform=ccrs.PlateCarree(),
                fontsize=8, horizontalalignment='left')

    # Plot the estimated source
    ax1.scatter([estimated_lon], [estimated_lat], color='red', s=150, marker='*',
               transform=ccrs.PlateCarree(), label='Estimated Source')

    # Add a colorbar
    sm = plt.cm.ScalarMappable(cmap='RdBu_r', norm=norm)
    cbar = plt.colorbar(sm, ax=ax1, pad=0.05, orientation='horizontal')
    cbar.set_label('Residual (seconds)')

    ax1.set_title('Source Localization with Residuals\n'
                 f'Estimated Position: ({estimated_lat:.4f}°, {estimated_lon:.4f}°)')

    # --- Second subplot: Residual plot ---
    # Create a simple plot to show residuals by station
    ax2.set_aspect('auto')  # Reset aspect ratio for this plot

    # Create bar plot of residuals
    bar_positions = np.arange(len(included_station_ids))
    colors = [plt.cm.RdBu_r(norm(res)) for res in residuals]

    # Plot bars
    bars = ax2.bar(bar_positions, residuals, color=colors, edgecolor='black')

    # Highlight outliers with pattern
    for i, is_out in enumerate(is_outlier):
        if is_out:
            bars[i].set_hatch('///')

    # Add zero line
    ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)

    # Add threshold lines
    mad = np.median(np.abs(residuals - np.median(residuals)))
    threshold = 3.5 * 1.4826 * mad
    ax2.axhline(y=threshold, color='red', linestyle='--', alpha=0.5, label=f'Outlier Threshold (±{threshold:.2f}s)')
    ax2.axhline(y=-threshold, color='red', linestyle='--', alpha=0.5)

    # Set labels
    ax2.set_xlabel('Station ID')
    ax2.set_ylabel('Residual (seconds)')
    ax2.set_title('Arrival Time Residuals by Station')
    ax2.set_xticks(bar_positions)
    ax2.set_xticklabels([f'Station {sid}' for sid in included_station_ids], rotation=45)
    ax2.grid(True, linestyle='--', alpha=0.7)
    ax2.legend()

    plt.tight_layout()
    plt.savefig('residual_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

    return residuals, is_outlier

# Call the function with your results
residuals, outliers = visualize_residuals(result, station_positions, arrival_times_seconds, ds, LAT_BOUNDS, LON_BOUNDS)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming `result` from least_squares and `station_positions`, `arrival_times_seconds`, and `ds` are in scope

# Extract residuals and Jacobian
residuals = result.fun
J = result.jac

# Number of observations and parameters
n_obs = residuals.size
n_params = result.x.size

# Compute reduced chi-squared
SSR = np.sum(residuals**2)
reduced_chi2 = SSR / (n_obs - n_params)

# Parameter covariance and standard errors
cov_params = np.linalg.inv(J.T @ J) * reduced_chi2
param_std = np.sqrt(np.diag(cov_params))

# Parameter names
param_names = ['latitude', 'longitude', 't0']

# Print summary table
print(f"Reduced χ²: {reduced_chi2:.3f}\n")
print("Parameter estimates with 1σ uncertainties:")
for name, val, err in zip(param_names, result.x, param_std):
    print(f"  {name:<10} = {val:.6f} ± {err:.6f}")

# Plot histogram of weighted residuals
plt.figure()
plt.hist(residuals, bins=10, edgecolor='black')
plt.xlabel('Weighted residuals')
plt.ylabel('Count')
plt.title('Histogram of Weighted Residuals')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Geod

# compute distances & azimuths from your fitted epicenter
lat_fit, lon_fit, _ = result.x
geod = Geod(ellps="WGS84")
dists, azs, res = [], [], result.fun

for (slat, slon), r in zip(station_positions, res):
    az12, az21, dist = geod.inv(lon_fit, lat_fit, slon, slat)
    dists.append(dist/1000)   # in km
    azs.append(az12)

plt.figure()
plt.scatter(dists, res, edgecolor='k')
plt.xlabel('Source–station distance (km)')
plt.ylabel('Weighted residual (s)')
plt.title('Residual vs. Distance')
plt.show()

plt.figure()
plt.scatter(azs, res, edgecolor='k')
plt.xlabel('Azimuth from event (°)')
plt.ylabel('Weighted residual (s)')
plt.title('Residual vs. Azimuth')
plt.show()


In [ ]:
STATIONS[0].get_pos(include_depth=True)